In [ ]:
import rasterio
import xarray as xr
import pandas 
import geopandas as gpd
import warnings
from shapely.geometry import mapping
import glob
from joblib import Parallel, delayed
import rioxarray as rxr
from category_encoders.james_stein import JamesSteinEncoder
import numpy as np
import rasterio
import os
import ee
import geemap
import gdal
from joblib import Parallel,delayed
from osgeo import gdal
from osgeo import ogr
from functools import reduce
from osgeo import gdalconst
import pandas as pd
import tqdm

In [ ]:
lis=glob.glob(r'F:\tif data\dtm*.tif')

def name(val):
    return(val.split('_')[1]+val.split('_')[4])
print(list(map(name,lis)))

vector=gpd.read_file(r'F:\Ind_boundary\India_Boundary.shp')
def func(i):
    raster=rxr.open_rasterio(i, mode = 'r', masked=True).squeeze()
    c=raster.rio.clip(vector.geometry.apply(mapping))
    nam=name(i)
    c.rio.to_raster(r'F:\tif data/'+nam+'_ind.tif')

In [ ]:
def clip(inp_loc,out_loc):
    raster=rxr.open_rasterio(inp_loc, mode = 'r', masked=True).squeeze()
    c=raster.rio.clip(vector.geometry.apply(mapping))
    c.rio.to_raster(out_loc)

In [ ]:
def reproject_raster(src_path, dst_path, dst_crs='EPSG:4326'):

    with rasterio.open(src_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(dst_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

In [ ]:
def resample(referenceFile,inputFile):
    reference = gdal.Open(referenceFile, 0)  # this opens the file in only reading mode
    referenceTrans = reference.GetGeoTransform()
    x_res = referenceTrans[1]
    y_res = -referenceTrans[5]  # make sure this value is positive
    outputFile=inputFile.split('.')[0]+inputFile.split('.')[1]+'_resample.tif'
    kwargs = {"format": "GTiff", "xRes": x_res, "yRes": y_res} #"resampleAlg":'mode'
    ds = gdal.Warp(outputFile, inputFile, **kwargs)

In [ ]:
ndsm = r"F:\tif data\paper1 tif\elev_aspect_slope.tif"
shp = r'F:\paper-1\paper1_data\India_COOLR.shp'
data = gdal.Open(ndsm, gdalconst.GA_ReadOnly)
geo_transform = data.GetGeoTransform()
#source_layer = data.GetLayer()
x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * data.RasterXSize
y_min = y_max + geo_transform[5] * data.RasterYSize
x_res = data.RasterXSize
y_res = data.RasterYSize
mb_v = ogr.Open(shp)
mb_l = mb_v.GetLayer()
pixel_width = geo_transform[1]
output =r"F:\tif data\paper1 tif\my.tif"
target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
band = target_ds.GetRasterBand(1)
NoData_value = -999999
band.SetNoDataValue(NoData_value)
band.FlushCache()
gdal.RasterizeLayer(target_ds, [1], mb_l,burn_values=[1])
target_ds = None

In [ ]:
ndsm=r"F:\tif data\paper1 tif\elev_aspect_slope.tif"
shp=r"F:\shapefiles\LANDSLIDE POLYGON\result_polygon.shp"
data = gdal.Open(ndsm, gdalconst.GA_ReadOnly)
geo_transform = data.GetGeoTransform()
source_layer = data.GetLayer()
x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * data.RasterXSize
y_min = y_max + geo_transform[5] * data.RasterYSize
x_res = data.RasterXSize
y_res = data.RasterYSize
mb_v = ogr.Open(shp)
mb_l = mb_v.GetLayer()
pixel_width = geo_transform[1]
output =r"F:\tif data\paper1 tif\polygon_raster.tif"
target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
band = target_ds.GetRasterBand(1)
NoData_value = -999999
band.SetNoDataValue(NoData_value)
band.FlushCache()
gdal.RasterizeLayer(target_ds, [1], mb_l,burn_values=[1],options =["ALL_TOUCHED=TRUE"])
target_ds = None

In [ ]:
#  burn=1
ndsm=r"F:\tif data\paper1 tif\elev_aspect_slope.tif"
data = gdal.Open(ndsm, gdalconst.GA_ReadOnly)
geo_transform = data.GetGeoTransform()
source_layer = data.GetLayer()
x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * data.RasterXSize
y_min = y_max + geo_transform[5] * data.RasterYSize
x_res = data.RasterXSize
y_res = data.RasterYSize
mb_v = ogr.Open(shp)
mb_l = mb_v.GetLayer()
pixel_width = geo_transform[1]
target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
band = target_ds.GetRasterBand(1)
NoData_value = -999999
band.SetNoDataValue(NoData_value)
band.FlushCache()
gdal.RasterizeLayer(target_ds, [1], mb_l,burn_values=[1])
target_ds = None

In [ ]:
file=xr.open_rasterio(r'F:\tif data\paper1 tif\elev_aspect_slope.tif')
band=file.descriptions

lis=[file[i].to_dataframe(name='{}'.format(band[i])).drop(['band'],axis=1) for i in range(len(file['band']))]
df=pd.concat(lis,axis=1,join='inner')


In [ ]:
## This is to check where the slope is null and the elevation is not null. The points lie on the edge and we can assume and drop them.

ddf=df[df['slope'].isna()]
gdf = geopandas.GeoDataFrame(ddf)
gdf.set_geometry(
    geopandas.points_from_xy(ddf['x'], ddf['y']),
    inplace=True, crs='EPSG:4326')
gdf.drop(['x', 'y'], axis=1, inplace=True)  # optional
gdf.to_file('r.shp')

In [ ]:
file['inv_point']=file['inv_point'].astype(int)
file['label']=(file['coolr landslide']) |(file['inventory_poly']) | (file['inv_point']) 
file.drop(['coolr landslide','inventory_poly','inv_point'],axis=1,inplace=True)
file.to_csv(r'F:\tif data\final_with_label.csv',index=False)

In [ ]:
file=pd.read_csv(r'F:\tif data\final_with_label.csv')
file.sort_values(by = ["y", "x"], ascending = [False, True],inplace=True)
file.loc[(file['sand_indtif_resample']==0),'sand_indtif_resample']=np.nan
file.loc[(file['silt_indtif_resample']==0),'silt_indtif_resample']=np.nan
file.loc[(file['clay_indtif_resample']==0),'clay_indtif_resample']=np.nan
file.interpolate(method='linear',limit_direction='both',inplace=True)

In [ ]:
file.loc[((file['aspect']<=22.5) | (file['aspect']>=337.5)),'Aspect']='n'
file.loc[((file['aspect']>22.5) & (file['aspect']<=67.5)),'Aspect']='ne'
file.loc[((file['aspect']>67.5) & (file['aspect']<=112.5)),'Aspect']='e'
file.loc[((file['aspect']>112.5) & (file['aspect']<=157.5)),'Aspect']='se'
file.loc[((file['aspect']>157.5) & (file['aspect']<=202.5)),'Aspect']='s'
file.loc[((file['aspect']>247.5) & (file['aspect']<=292.5)),'Aspect']='w'
file.loc[((file['aspect']>292.5) & (file['aspect']<=337.5)),'Aspect']='nw'
file.loc[((file['aspect']>202.5) & (file['aspect']<=247.5)),'Aspect']='sw'
JSE_encoder = JamesSteinEncoder()
train_jse=pd.DataFrame()
train_jse['as']= JSE_encoder.fit_transform(file.Aspect, file.label)
file1=pd.concat([file,train_jse], axis=1)
file1.drop(['Aspect','aspect'],inplace=True,axis=1)